## Introdução

A extração e tratamento dos dados presentes nos bancos de dados do DATASUS podem fornecer informações precisas e orientar condutas e políticas públicas. O Sistema de Informações Ambulatoriais (SIA) apresenta uma estrutura e conjunto de dados que, agreagados da forma correta, podem ser úteis tanto cientificamente quanto estrategicamente, como Ferré et al. demonstrou e exemplificou métodos para extrair coortes dos bancos de dados do SIA do DATASUS (e de outros sistemas), o que torna possível a análise de diversos elementos, dentre eles, a análise da dispensação de medicamentos, quantitativo de usuários desses medicamentos e o valor gasto.

## Extração

In [ ]:
#=================[PACOTES]==================

install.packages("devtools") # Para instalar pacotes pelo Github
devtools::install_github("rfsaldanha/microdatasus") # Para manipular os bancos de dados
install.packages("gsubfn") # Para alterar dados no DataFrame
install.packages("RCurl") # Função getURL
install.packages("stringr") # lpad str_pad
install.packages("readr") # Para ler CSV diretamente do GitHub
install.packages("forecast") # Para ARIMA
install.packages("tseries") # Para regressão

# Carregando os pacotes

library(microdatasus)
library(gsubfn)
library(stringr)
library(RCurl)
library(readr)
library(forecast)
library(tseries)

#============================================

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Skipping install of 'microdatasus' from a github remote, the SHA1 (d59f8a5f) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: prot

O que será feito:

1. Dados armazenados em arquivos `.dbc` no repositório FTP do DATASUS
2. Download dos dados por meio do `microdatasus`
3. Armazenamento temporário dos dados localmente
4. Inclusão de variáveis selecionadas no dataframe de importação
5. Filtrar o dataframe de acordo com CID-10 e SIGTAP
6. Unir os bancos de dados do SIA-PA e SIA-AM

### Extração DATASUS SIA-PA

A função `fetch_datasus` é responsável pelo download de dados do site do DATASUS, criando um `data.frame` como resultado. O manual para a função está na [Wiki do projeto](https://github.com/rfsaldanha/microdatasus/wiki/fetch_datasus). Com a função, foi realizado o download do banco de dados de acordo com as variáveis permitidas pela função.

In [ ]:
#=================[DADOS SIA-PA]=============

siapa_go <- fetch_datasus(
                       year_start = 2023, year_end = 2023,
                       month_start = 11, month_end = 12,
                       uf = "GO",
                       information_system = "SIA-PA")

#============================================

ℹ Your local Internet connection seems to be ok.

ℹ DataSUS FTP server seems to be up and reachable.

ℹ Starting download...



### Extração DATASUS SIA-AM

O processo foi repetido para outro banco de dados de interesse.

In [ ]:
#=================[DADOS SIA-AM]=============

siaam_go <- fetch_datasus(
                       year_start = 2023, year_end = 2023,
                       month_start = 11, month_end = 12,
                       uf = "GO",
                       vars = c("AP_AUTORIZ", "AP_PRIPAL", "AP_CIDPRI", "AP_CNSPCN"),
                       information_system = "SIA-AM")

#============================================

ℹ Your local Internet connection seems to be ok.

ℹ DataSUS FTP server seems to be up and reachable.

ℹ Starting download...



### Verificação da Extração

O controle da ordem de manipulação dos arquivos segundo estado e mês é fundamental na carga.

A função `expand.grid` é empregada para gerar as combinações de UF, ano e mês
que integram o nome dos arquivos a serem manipulados.

In [ ]:
#=================[VERIFICAÇÃO]==============

ano=23:23 # Igual em year_star e year_end
mes=str_pad(11:12, 2, pad="0") # Igual em month_start e month_end
url="ftp://ftp.datasus.gov.br/dissemin/publicos/SIASUS/200801_/Dados/"

# Lista arquivos dbc do diretorio FTP
aux <-
  getURL(
    url,
    verbose = TRUE,
    ftp.use.epsv = FALSE,
    dirlistonly = TRUE,
    crlf = TRUE
  )

listadbc=strsplit(aux, "\r*\n")[[1]]

#length(listadbc) # Número de arquivos

#listadbc[sample(1:length(listadbc),20, replace = TRUE)]

uf='GO'

aux=as.matrix(expand.grid(uf,ano,mes))
ufaamm=sort(paste0(
    aux[,1],
    aux[,2],
    aux[,3]
))
ufaamm

#============================================

[1] "GO2311" "GO2312"

## Processamento PA

Para o processamento, primeiro foram definidos os filtos com base no CID-10 e o procedimento descrito no SIGTAP. Após isso, foram criadas tabelas vazias no formato `data.frame` contendo as variáveis desejadas. As variáveis desejadas foram analisadas e selecionadas a partir do [dicionário de dados](ftp://ftp.datasus.gov.br/dissemin/publicos/SIASUS/200801_/Doc/.) do Sistema de Informações Ambulatoriais do SUS.

Eventualmente arquivos do mês podem ultrapassar cerca de dois milhões de registros e, dessa forma, são fragmentados e mais de um arquivo. Por exemplo, *PASP2301a.dbc*, *PASP2301b.dbc* e *PASP2301c.dbc*. Por isso é utilizada a estrutura de repetição `for`.

Assim, os filtros de CID-10 e procedimento SIGTAP foram usados logo após a carga para reduzir a ocupação na memória e, portanto, o recurso computacional necessário.

Foram estabelecidos filtros segundo a doença selecionada em `PA_CIDPRI %in% cids` e procedimento em `PA_PROC_ID %in% sigtap`.

Adicionalmente, o vetor de atributos foi aplicado a fim de desprezar os demais.

In [ ]:
#=================[FILTROS]=================

cids = c(
  "K500",
  "K501",
  "K508"
) # CID-10

sigtap = c(
  "0604380011", # ADALIMUMABE 40 MG INJETÁVEL (FRASCO AMPOLA)
  "0604380127", # ADALIMUMABE 40 MG INJETAVEL (POR SERINGA PREENCHIDA)
  "0604380135", # ADALIMUMABE 40 MG INJETÁVEL (POR SERINGA PREENCHIDA) (BIOSSIMILAR B)
  "0604380046", # INFLIXIMABE 10 MG/ML INJETAVEL (POR FRASCO-AMPOLA COM 10 ML)
  "0604380054", # INFLIXIMABE 10 MG/ML INJETAVEL (POR FRASCO-AMPOLA COM 10 ML)
  "0604380119", # INFLIXIMABE 10 MG /ML INJETÁVEL (POR FRASCO-AMPOLA COM 10 ML) (BIOSSIMILAR A)
  "0604380070"  # CERTOLIZUMABE PEGOL 200 MG/ML INJETÁVEL (POR SERINGA PREENCHIDA)
) # Biológicos

#=================[PROCESSAMENTO-PA]========

pa_estrutura=data.frame(
  PA_AUTORIZ = numeric(),
  PA_CMP = numeric(),
  PA_MVM = numeric(),
  PA_CIDPRI = character(),
  PA_CIDSEC = character(),
  PA_PROC_ID = character(),
  PA_QTDAPR = numeric(),
  PA_SEXO = character(),
  PA_IDADE = numeric(),
  PA_MUNPCN = numeric(),
  uf_processamento  = character()
)

# Filtra apenas arquivos contendo o respectivo UFAAMM

listadbc_pa=subset(
      listadbc,
      grepl(paste0("^PA",ufaamm[i]), listadbc)
    )

pa=pa_estrutura

# Incorpora o arquivo PA

    for (j in 1:length(listadbc_pa)) {
      aux=subset(siapa_go[,c(
          "PA_AUTORIZ","PA_CMP","PA_MVM" ,"PA_CIDPRI", "PA_CIDSEC",
          "PA_PROC_ID", "PA_QTDAPR","PA_SEXO", "PA_IDADE", "PA_MUNPCN"
        )],
        PA_CIDPRI %in% cids &
        PA_PROC_ID %in% sigtap
      )

      if (nrow(aux)==0) {
        aux=pa
      } else {
        aux$uf_processamento = uf
        pa=rbind(pa,aux)
      }

    }

head(pa)

#============================================

,PA_AUTORIZ,PA_CMP,PA_MVM,PA_CIDPRI,PA_CIDSEC,PA_PROC_ID,PA_QTDAPR,PA_SEXO,PA_IDADE,PA_MUNPCN,uf_processamento
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
483489...483489,5223203448661,202311,202311,K500,0000,0604380054,5,M,019,522045,GO
483491...483491,5223203476447,202311,202311,K500,0000,0604380054,4,M,059,520110,GO
483492...483492,5223203493057,202311,202311,K500,0000,0604380054,4,F,067,521890,GO
483493...483493,5223204359263,202311,202311,K501,0000,0604380054,0,M,030,521190,GO
483495...483495,5223204340838,202311,202311,K500,0000,0604380054,2,M,011,520870,GO
483496...483496,5223203515948,202311,202311,K500,0000,0604380054,4,M,057,520110,GO


## Processamento AM e união com PA

Por meio do número de autorização, em geral, **PA_AUTORIZ** ou **AP_AUTORIZ**, é possível completar o arquivo principal. A tarefa é fundamental para computar o número de usuários com quantidade aprovada de dado procedimento, sobretudo a partir do arquivo de medicamentos de prefixo AM.

A fim de agilizar o processamento, os caracteres especiais do Cartão Nacional de Saúde (CNS) criptografado são substituídos por números.

A junção foi realizada segundo código da autorização. Note a incorporação do atributo **AP_CNSPCN**. Se mais atributos forem necessários basta incorporar ao vetor do arquivo AM.


In [ ]:
#=================[PROCESSAMENTO-AM]========

am_estrutura=data.frame(
  AP_AUTORIZ = numeric(),
  AP_PRIPAL = character(),
  AP_CIDPRI = character(),
  AP_CNSPCN = character()
)

# Filtra apenas arquivos contendo o respectivo UFAAMM

listadbc_am=subset(
      listadbc,
      grepl(paste0("^AM",ufaamm[i]), listadbc)
    )


am=am_estrutura

# Incorpora o arquivo AM

    if (length(listadbc_am) > 0) {
    for (j in 1:length(listadbc_am)) {
      aux=subset(siaam_go[,c(
          "AP_AUTORIZ", "AP_PRIPAL",
          "AP_CIDPRI",  "AP_CNSPCN"
        )],
        AP_CIDPRI %in% cids &
        AP_PRIPAL %in% sigtap
      )
      am=rbind(am,aux)

    }

#=================[CARACTERES]===============

      am2=unique(am[,c("AP_AUTORIZ","AP_CNSPCN")])

      am2$AP_CNSPCN=gsubfn(".",
                         list("{" = "0", "}" = "9", "~" = "5",
                              "\177" = "7", "Ç" = "6", "ä" = "8",
                              "ü" = "4", "é" = "1", "|" = "2", "â" = "3"
                              ),
                         iconv(am2$AP_CNSPCN, "CP861", "UTF-8")
                         )

#=================[PA+AM]====================

      aux = merge(
        pa, am2,
        by.x=c("PA_AUTORIZ"), # Possibilita a união
        by.y=c("AP_AUTORIZ"), # Possibilita a união
        all.x = TRUE
        )
    } else {
      aux=pa
      aux$AP_CNSPCN = NA
    }

paam_estrutura=data.frame(
    PA_AUTORIZ = numeric(),
    PA_CMP = numeric(),
    PA_MVM = numeric(),
    PA_CIDPRI = character(),
    PA_CIDSEC = character(),
    PA_PROC_ID = character(),
    PA_QTDAPR = numeric(),
    PA_SEXO = character(),
    PA_IDADE = numeric(),
    PA_MUNPCN = numeric(),
    uf_processamento  = character(),
    AP_CNSPCN = character()
)

paam=paam_estrutura
paam=rbind(paam,aux)
head(paam)

#============================================


,PA_AUTORIZ,PA_CMP,PA_MVM,PA_CIDPRI,PA_CIDSEC,PA_PROC_ID,PA_QTDAPR,PA_SEXO,PA_IDADE,PA_MUNPCN,uf_processamento,AP_CNSPCN
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,5223202118596,202311,202311,K500,0000,0604380054,0,M,038,521850,GO,100008039594104
2,5223202126538,202311,202311,K500,0000,0604380054,0,M,042,521880,GO,109706638936091
3,5223202127605,202311,202311,K508,0000,0604380119,5,M,027,520860,GO,105206489989080
4,5223202149209,202311,202311,K500,0000,0604380119,0,M,044,520410,GO,100703884038076
5,5223202150782,202311,202311,K508,0000,0604380054,0,M,023,520870,GO,383007395806405
6,5223202151508,202311,202311,K500,0000,0604380119,3,M,016,520110,GO,102900033460229


## Exportar CSV

 Finalmente, foi obtido o `data.frame` com todas variáveis de interesse adicionadas e filtradas de acordo com os filtros prévios. O resultado foi exportado no formato `.csv` para posterior modelagem e análise dos dados.

In [ ]:
#=================[CSV]======================

write.csv(paam,paste0("sia_pa_",uf,".csv"))

#============================================

## Modelagem dos Dados

### Agregação anual para avaliação da quantidade aprovada e número de usuários

O arquivo .csv gerado foi, então, carregado para a análise. Foi gerada a coluna **ano**, a partir dos 4 primeiros dígitos da coluna **PA_CMP**.

In [ ]:
#=================[CARREGAR CSV]=============

options(scipen=999) # Para evitar notação científica
paam_atendimento=read.csv("/content/sia_pa_GO.csv") # Caminho para o .csv
paam_atendimento$ano=substr(paam_atendimento$PA_CMP,1,4)
head(paam_atendimento)

#============================================

,X,PA_AUTORIZ,PA_CMP,PA_MVM,PA_CIDPRI,PA_CIDSEC,PA_PROC_ID,PA_QTDAPR,PA_SEXO,PA_IDADE,PA_MUNPCN,uf_processamento,AP_CNSPCN,ano
,<int>,<dbl>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<chr>
1,1,5223202118596,202311,202311,K500,0,604380054,0,M,38,521850,GO,100008039594104,2023
2,2,5223202126538,202311,202311,K500,0,604380054,0,M,42,521880,GO,109706638936091,2023
3,3,5223202127605,202311,202311,K508,0,604380119,5,M,27,520860,GO,105206489989080,2023
4,4,5223202149209,202311,202311,K500,0,604380119,0,M,44,520410,GO,100703884038076,2023
5,5,5223202150782,202311,202311,K508,0,604380054,0,M,23,520870,GO,383007395806405,2023
6,6,5223202151508,202311,202311,K500,0,604380119,3,M,16,520110,GO,102900033460229,2023


Uma sigla foi atribuída para cada código de medicamento extraído do SIGTAP.

In [ ]:
#=================[COLUNA MEDICAMENTO]========

paam_atendimento$medicamento=""
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380127'] <- 'ADA'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380011'] <- 'ADA'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380135'] <- 'ADA'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380070'] <- 'CER'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380119'] <- 'INF'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380046'] <- 'INF'
paam_atendimento$medicamento[paam_atendimento$PA_PROC_ID == '604380054'] <- 'INF'

head(paam_atendimento)

#============================================

,X,PA_AUTORIZ,PA_CMP,PA_MVM,PA_CIDPRI,PA_CIDSEC,PA_PROC_ID,PA_QTDAPR,PA_SEXO,PA_IDADE,PA_MUNPCN,uf_processamento,AP_CNSPCN,ano,medicamento
,<int>,<dbl>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
1,1,5223202118596,202311,202311,K500,0,604380054,0,M,38,521850,GO,100008039594104,2023,INF
2,2,5223202126538,202311,202311,K500,0,604380054,0,M,42,521880,GO,109706638936091,2023,INF
3,3,5223202127605,202311,202311,K508,0,604380119,5,M,27,520860,GO,105206489989080,2023,INF
4,4,5223202149209,202311,202311,K500,0,604380119,0,M,44,520410,GO,100703884038076,2023,INF
5,5,5223202150782,202311,202311,K508,0,604380054,0,M,23,520870,GO,383007395806405,2023,INF
6,6,5223202151508,202311,202311,K500,0,604380119,3,M,16,520110,GO,102900033460229,2023,INF


A tabela foi carregada na variável paam_atendimento, na qual cada linha representa um atendimento. É necessário extrair o ano do mês de competência **PA_CMP** em função do procedimento **SIGTAP PA_PROC_ID**, somando a quantidade aprovada **PA_QTDAPR** e contando o número de usuários distintos **AP_CNSPCN**.

In [ ]:
#=================[QUANTIDADE APROVADA]======

quantidade=aggregate(
  PA_QTDAPR ~ ano + medicamento,
  data = paam_atendimento,
  FUN = sum,
  na.rm = TRUE
  )
head(quantidade,20)

#============================================

,ano,medicamento,PA_QTDAPR
,<chr>,<chr>,<int>
1,2023,ADA,0
2,2023,CER,46
3,2023,INF,1672


In [ ]:
#=================[USUÁRIOS DISTINTOS]=======

usuarios=aggregate(
  AP_CNSPCN ~ ano + medicamento,
  data = paam_atendimento,
  function(x) length(unique(x))
  )
head(usuarios,20)

#============================================

,ano,medicamento,AP_CNSPCN
,<chr>,<chr>,<int>
1,2023,ADA,1
2,2023,CER,11
3,2023,INF,280


In [ ]:
#=================[USUÁRIOS+MEDICAMENTOS]====

paam_anual = merge(
  usuarios, quantidade,
  by.x=c("ano","medicamento"),
  by.y=c("ano","medicamento"),
  all.x = TRUE
  )
paam_anual

#============================================

ano,medicamento,AP_CNSPCN,PA_QTDAPR
<chr>,<chr>,<int>,<int>
2023,ADA,1,0
2023,CER,11,46
2023,INF,280,1672


### Cálculo do custo dos medicamentos

Uma tabela secundária com os valores unitários dos medicamentos pré-selecionados foi incluída. Os valores foram obtidos do [Banco de Preços em Saúde](https://www.gov.br/saude/pt-br/acesso-a-informacao/banco-de-precos). Com isso, foi possível multiplicar o valor médio unitário pela quantidade aprovada **PA_QTDAPR**.

In [ ]:
#=================[CSV VALOR UNITÁRIO]=======

valor=read_csv("https://raw.githubusercontent.com/victorcords/info-siasus/main/bps_2018_2023/bps_valor_unitario.csv")
valor

#============================================

Após isso, os valores atribuídos foram associados com os dados de mundo real.

In [ ]:
#=================[VALOR+DIFUSÃO]============

valor=read_csv("https://raw.githubusercontent.com/victorcords/info-siasus/main/bps_2018_2023/bps_valor_unitario_tranposto.csv")

paam_anual_valor = merge(
        paam_anual, valor,
        by.x=c("ano","medicamento"),
        by.y=c("ano","medicamento"),
        all.x = TRUE
      )

# Percentual de difusão ao ano do número de usuários

paam_anual_valor=transform(paam_anual_valor, usuario_perc = ave(AP_CNSPCN, ano, FUN = prop.table))
paam_anual_valor$usuario_perc=round(paam_anual_valor$usuario_perc*100,1)

# Produto entre total aprovado e valor unitário

paam_anual_valor$total=round(paam_anual_valor$PA_QTDAPR*paam_anual_valor$unidade)

# Percentual de difusão ao ano do total

paam_anual_valor=transform(paam_anual_valor, total_perc = ave(total, ano, FUN = prop.table))
paam_anual_valor$total_perc=round(paam_anual_valor$total_perc*100,1)

paam_anual_valor$unidade=round(paam_anual_valor$unidade,2)

paam_anual_valor

#============================================

### Cálculo da difusão de uso

Vamos explorar visualmente a difusão do percentual de uso com `ggplot`.

In [ ]:
#=================[GRÁFICO DIFUSÃO]============

ggplot(
  paam_anual_valor,
  aes(
    fill = medicamento,
    y = usuario_perc,
    x = ano,
    label = usuario_perc
  )
) + geom_bar(stat = "identity") +
  xlab('ano') +
  ylab('usuarios (%)') +
  geom_text(size = 3, position = position_stack(vjust = 0.5)) +
  coord_flip() +
  theme(
    legend.text = element_text(size = rel(1.3)),
    axis.title.x = element_text(size = rel(1.3)),
    axis.title.y = element_text(size = rel(1.3)),
    axis.text.x = element_text(size = rel(1.3)),
    axis.text.y = element_text(size = rel(1.3))
    )

#============================================

Além disso, também é possível calcular o valor mensal total gasto com as medicações.

In [ ]:
#=================[VALOR MENSAL]=============

paam_mensal_valor = merge(
  paam_atendimento, valor,
  by.x=c("ano","medicamento"),
  by.y=c("ano","medicamento"),
  all.x = TRUE
  )

# Produto entre total aprovado e valor unitário

paam_mensal_valor$total=round(paam_mensal_valor$PA_QTDAPR*paam_mensal_valor$unidade)
total=aggregate(
  total ~ PA_CMP + medicamento,
  data = subset(
    paam_mensal_valor,
    medicamento == 'INF'
    ),
  FUN = sum,
  na.rm = TRUE
  )

head(total,20)

#============================================

Os dados de mundo real obtidos podem ser processados com técnicas de regressão ou séries temporais. O método ARIMA permite afinar o modelo especificando ordens de autoregressão, diferenciamento e média móvel, bem como contrapartidas sazonais a esses componentes.

Foi realizado o método ARIMA para variável **total**.

In [ ]:
#=================[ARIMA]====================

arima_data=forecast(auto.arima(
    total$total,
    allowmean = TRUE
  ), h = 60)

# Projeção para o valor total (R$)

plot(
  arima_data,
  xlab = "",
  ylab = "Total (R$)",
  main = "",
  las = 3  ,
  xaxt = 'n'
)

axis(1,
     at = 1:(nrow(total)+61),
     labels = 1:(nrow(total)+61),
     las = 3)

arima_data=as.data.frame(arima_data)
colnames(arima_data)=c("previsão","min80","min95","max80","max95")
tail(arima_data,60)

#============================================

E também, foi realizado o método ARIMA para variável **usuários**, filtrado pelo medicamento "INF".

In [ ]:
#=================[ARIMA]====================

usuarios=aggregate(
  AP_CNSPCN ~ PA_CMP + medicamento,
  data = subset(
      paam_atendimento,
      medicamento == 'INF'
    ),
  function(x) length(unique(x))
)
tail(usuarios,20)

arima_data=forecast(auto.arima(
    usuarios$AP_CNSPCN,
    allowmean = TRUE
  ), h = 60)

# Projeção para o número de usuários filtrados pela variável medicamento

plot(
  arima_data,
  xlab = "",
  ylab = "Usuários",
  main = "",
  las = 3  ,
  xaxt = 'n'
)

axis(1,
     at = 1:(nrow(total)+61),
     labels = 1:(nrow(total)+61),
     las = 3)

arima_data=as.data.frame(arima_data)
colnames(arima_data)=c("previsão","min80","min95","max80","max95")
tail(arima_data,60)

#============================================

## Referências

CAROLINA ANDRADE OLIVEIRA DIBAI; JULIANA ALVARES-TEODORO; FELIPE FERRÉ; MARIANO RUAS, C. Avaliação da eficiência na aquisição e distribuição de medicamentos do Componente Básico da Assistência Farmacêutica. JORNAL DE ASSISTÊNCIA FARMACÊUTICA E FARMACOECONOMIA, [S. l.], v. 8, n. 4, 2023. DOI: 10.22563/2525-7323.2023.v8.n.4.p.23-36. Disponível em: https://ojs.jaff.org.br/ojs/index.php/jaff/article/view/642. Acesso em: 23 mai. 2024.

FERRÉ, F. Produto 3 - Relatório técnico contendo análise da projeção do quantitativo de usuários, unidades dispensadas e valor gasto apresentados na sala de situação aberta de medicamentos biológicos cujo acesso é regulado por PCDT. Brasil: RStudio/RPubs, 2022. Disponível em: <https://rpubs.com/labxss/858478>. Acesso em: 23 mai. 2024.

FERRÉ, F.; DE OLIVEIRA, G.; DE QUEIROZ, M.; GONÇALVES, F.. Sala de Situação aberta com dados administrativos para gestão de Protocolos Clínicos e Diretrizes Terapêuticas de tecnologias providas pelo SUS. In: SIMPÓSIO BRASILEIRO DE COMPUTAÇÃO APLICADA À SAÚDE (SBCAS), 20. , 2020, Evento Online. Anais [...]. Porto Alegre: Sociedade Brasileira de Computação, 2020 . p. 392-403. ISSN 2763-8952. DOI: https://doi.org/10.5753/sbcas.2020.11530.

FERRÉ, F. Curso de linguagem R com dados do SUS. Disponível em: <https://github.com/labxss/curso_r>. Acesso em: 23 mai. 2024.

FERRÉ, F. Sala Aberta de Inteligência em Saúde - Protocolos Clínicos e Diretrizes Terapêuticas. Disponível em: <https://github.com/labxss/sabeis_pcdt>. Acesso em: 21 mai. 2024.


SALDANHA, R. DE F.; BASTOS, R. R.; BARCELLOS, C.. Microdatasus: pacote para download e pré-processamento de microdados do Departamento de Informática do SUS (DATASUS). Cadernos de Saúde Pública, v. 35, n. 9, p. e00032419, 2019.